In [2]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')

# display(autentica)


login = str(autentica['login'][1])
senha = str(autentica['senha'][1])

indisponivel = "indisponivel"

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [4]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element_by_id('id_elemento_despesa')
    options = [x for x in valor_ed.find_elements_by_tag_name('option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [5]:
def get_qtd():
    
    abrir_suap()
    
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    descricoes = []
    quantidades = []
    valores = []
    elemento = []
    
    try:
        pag = 1
        saida = True
        while saida:
                
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element_by_tag_name('td')
                
            if celula:
                lista = nav.find_element_by_tag_name('table')
                td = lista.find_elements_by_tag_name('td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        print('acabaram as páginas')
        
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])
        
    for ed in range(3, len(tabela), 7):
        elemento.append(tabela[ed])

    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela['ed'] = elemento
    
    df_tabela.to_excel('relatorio/relatorio.xlsx', index=False)
    display(df_tabela)
    
    nav.quit()


In [6]:
get_qtd()

acabaram as páginas


,codigo,descricao,qtd,vlr_total,vlr_unt,ed
0,30.07.2,"AÇÚCAR CRISTAL, EMBALAGEM C/ 1 KG.",101,202.00,2.00,07 - GENEROS DE ALIMENTACAO
1,30.07.252,"ADOÇANTE LÍQUIDO, 100 ML",26,100.90,3.88,07 - GENEROS DE ALIMENTACAO
2,30.07.43,"CAFÉ, EMBAL. A VÁCUO, C/ 250G.",177,805.35,4.55,07 - GENEROS DE ALIMENTACAO
3,30.11.115,"HERBICIDA, GLIFOSATO 480 G/L.",30,814.80,27.16,11 - MATERIAL QUIMICO
4,30.14.156,BRAÇADEIRA ELASTICA DE CAPITÃO.,1,17.05,17.05,14 - MATERIAL EDUCATIVO E ESPORTIVO
...,...,...,...,...,...,...
405,30.42.974,"CHAVE FENDA, MATERIAL HASTE AÇO VANÁDIO TREFIL...",14,194.32,13.88,42 - FERRAMENTAS
406,30.42.978,"CHAVE DE PHILLIPS, HASTE EM AÇO VANÁDIO, CABO ...",11,53.35,4.85,42 - FERRAMENTAS
407,30.42.990,"SUPORTE PARA FERRO DE SOLDA, BASE EM CHAPA MET...",12,168.00,14.00,42 - FERRAMENTAS
408,30.44.48,"PROTETOR CRACHÁ, NOME BOLSA DE CRACHA",41,28.70,0.70,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
